In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
 
import keras
import keras.backend as K
from keras.layers import Input, Convolution2D, Activation, MaxPooling2D, Dense, BatchNormalization, Dropout
from keras.layers.core import Flatten
from keras.optimizers import SGD
from keras.models import Model
from keras.utils import np_utils
from keras.constraints import maxnorm
from keras.regularizers import l2
from keras.callbacks import LearningRateScheduler
from keras.layers import BatchNormalization

from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate

In [18]:
# layer by layer pre-training models 

# Layer 1
input_num = Input(shape=(49,))
distorted_input1 = Dropout(.1)(input_num)
encoded1 = Dense(100, activation = 'relu')(input_num)
encoded1_bn = BatchNormalization()(encoded1)
decoded1 = Dense(49, activation = 'relu')(encoded1_bn)

autoencoder1 = Model(inputs = input_num, outputs = decoded1)
encoder1 = Model(inputs = input_num, outputs = encoded1_bn)

# Layer 2
encoded1_input = Input(shape = (100,))
distorted_input2 = Dropout(.2)(encoded1_input)
encoded2 = Dense(50, activation = 'relu')(distorted_input2)
encoded2_bn = BatchNormalization()(encoded2)
decoded2 = Dense(100, activation = 'relu')(encoded2_bn)

autoencoder2 = Model(inputs = encoded1_input, outputs = decoded2)
encoder2 = Model(inputs = encoded1_input, outputs = encoded2_bn)

# Layer 3 - which we won't end up fitting in the interest of time
encoded2_input = Input(shape = (50,))
distorted_input3 = Dropout(.3)(encoded2_input)
encoded3 = Dense(4, activation = 'relu')(distorted_input3)
encoded3_bn = BatchNormalization()(encoded3)
decoded3 = Dense(64, activation = 'relu')(encoded3_bn)

autoencoder3 = Model(inputs = encoded2_input, outputs = decoded3)
encoder3 = Model(inputs = encoded2_input, outputs = encoded3_bn)


# Deep Autoencoder ---------------------------------------------
encoded1_da = Dense(100, activation = 'sigmoid')(input_num)
encoded1_da_bn = BatchNormalization()(encoded1_da)
encoded2_da = Dense(50, activation = 'sigmoid')(encoded1_da_bn)
encoded2_da_bn = BatchNormalization()(encoded2_da)
encoded3_da = Dense(4, activation = 'sigmoid')(encoded2_da_bn)
encoded3_da_bn = BatchNormalization()(encoded3_da)
decoded3_da = Dense(50, activation = 'sigmoid')(encoded3_da_bn)
decoded2_da = Dense(100, activation = 'sigmoid')(decoded3_da)
decoded1_da = Dense(49, activation = 'sigmoid')(decoded2_da)

deep_autoencoder = Model(inputs = input_num, outputs = decoded1_da)

# Not as Deep Autoencoder
nad_encoded1_da = Dense(100, activation = 'sigmoid')(input_num)
nad_encoded1_da_bn = BatchNormalization()(nad_encoded1_da)
nad_encoded2_da = Dense(50, activation = 'sigmoid')(nad_encoded1_da_bn)
nad_encoded2_da_bn = BatchNormalization()(nad_encoded2_da)
nad_decoded2_da = Dense(100, activation = 'sigmoid')(nad_encoded2_da_bn)
nad_decoded1_da = Dense(49, activation = 'sigmoid')(nad_decoded2_da)

nad_deep_autoencoder = Model(inputs = input_num, outputs = nad_decoded1_da)

In [19]:
lr = 0.0001
sgd1 = keras.optimizers.Adam(learning_rate=lr)
sgd2 = keras.optimizers.Adam(learning_rate=lr)
sgd3 = keras.optimizers.Adam(learning_rate=lr)

autoencoder1.compile(loss='mean_squared_error', optimizer = sgd1)
autoencoder2.compile(loss='mean_squared_error', optimizer = sgd2)
autoencoder3.compile(loss='mean_squared_error', optimizer = sgd3)

encoder1.compile(loss='mean_squared_error', optimizer = sgd1)
encoder2.compile(loss='mean_squared_error', optimizer = sgd1)
encoder3.compile(loss='mean_squared_error', optimizer = sgd1)

deep_autoencoder.compile(loss='mean_squared_error', optimizer = sgd1)
nad_deep_autoencoder.compile(loss='mean_squared_error', optimizer = sgd1)


In [ ]:
autoencoder1.fit(training_inputs, training_inputs,
                nb_epoch = 8, batch_size = 512,
                validation_split = 0.30,
                shuffle = True)

In [ ]:
first_layer_code = encoder1.predict(training_inputs)
print(first_layer_code.shape)

In [ ]:
autoencoder2.fit(first_layer_code, first_layer_code,
                nb_epoch = 8, batch_size = 512,
                validation_split = 0.25,
                shuffle = True)

In [ ]:
# Setting up the weights of the not-as-deep autoencoder
nad_deep_autoencoder.layers[1].set_weights(autoencoder1.layers[2].get_weights()) # first dense layer
nad_deep_autoencoder.layers[2].set_weights(autoencoder1.layers[3].get_weights()) # first bn layer
nad_deep_autoencoder.layers[3].set_weights(autoencoder2.layers[2].get_weights()) # second dense layer
nad_deep_autoencoder.layers[4].set_weights(autoencoder2.layers[3].get_weights()) # second bn layer
nad_deep_autoencoder.layers[5].set_weights(autoencoder2.layers[4].get_weights()) # second decoder
nad_deep_autoencoder.layers[6].set_weights(autoencoder1.layers[4].get_weights()) # third decoder

---

In [62]:
#merged_latent.to_pickle('merged_latent49.pkl')
merged_latent = pd.read_pickle("merged_latent49.pkl")

# merged data 1
firm_col = merged_latent.columns.tolist()[18:18+49]
latent_col = merged_latent.columns.tolist()[18+49+1:]
print(len(firm_col), len(latent_col))

49 49


In [63]:
df_train = merged_latent[merged_latent.index < pd.Period((str(1995)+"-1"),freq='M')]
df_val = merged_latent[(merged_latent.index >= pd.Period((str(1995)+"-1"),freq='M')) & (merged_latent.index < pd.Period((str(2000)+"-1"),freq='M'))]
df_test = merged_latent[merged_latent.index >= pd.Period((str(2000)+"-1"),freq='M')]

merged_train_firm = df_train[firm_col]
merged_train_latent = df_train[latent_col]
merged_train_y = df_train['predicted_return']
print(merged_train_firm.shape, merged_train_latent.shape, merged_train_y.shape)

merged_val_firm = df_val[firm_col]
merged_val_latent = df_val[latent_col]
merged_val_y = df_val['predicted_return']
print(merged_val_firm.shape, merged_val_latent.shape, merged_val_y.shape)

merged_test_firm = df_test[firm_col]
merged_test_latent = df_test[latent_col]
merged_test_y = df_test['predicted_return']
print(merged_test_firm.shape, merged_test_latent.shape, merged_test_y.shape)

(88347, 49) (88347, 49) (88347,)
(31851, 49) (31851, 49) (31851,)
(90007, 49) (90007, 49) (90007,)


In [113]:
def Autoencoder():
    input_img = Input(shape=(49, ))
    encoded1 = Dense(6, activation='sigmoid')(input_img)
    decoded1 = Dense(49, activation='sigmoid')(encoded1)
    
    autoencoder = Model(inputs=input_img, outputs=decoded1)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    autoencoder.compile(loss='mean_squared_error', optimizer=opt, metrics=[my_metric_fn])
    return autoencoder

autoencoder = Autoencoder()

In [104]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

def multi_input_model():
    l1_reg = 0.0
    l2_reg = 0.0
    dropout = 0.0
    lr = 0.0001
        
    # Define the hidden layers 
    input1 = Input(shape=(49,))
    x1 = Dense(6, activation='relu')(input1)
        
    # Input 2
    input2 = Input(shape=(49,))
    x2 = Dense(32, activation='relu',)(input2)
    x2 = Dense(16, activation='relu',)(x2)
    x2 = Dense(6, activation='relu',)(x2)
        
    merged = Multiply()([x1, x2])
    output = Dense(1, activation='linear')(merged)
 
    # Create the model 
    model = Model(inputs=[input1, input2], outputs=output)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
    return model

In [110]:
import random 

seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
 
    model = multi_input_model()
    model.layers[4].set_weights(autoencoder1.layers[1].get_weights())
    model.layers[4].trainable = False
    model.fit([merged_train_latent, merged_train_firm], merged_train_y,
                   epochs=50,
                   validation_data=([merged_val_latent, merged_val_firm], merged_val_y),
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(model.evaluate([merged_test_latent, merged_test_firm], merged_test_y))
    y_hat = model.predict([merged_test_latent, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

Epoch 1/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0125 - my_metric_fn: -0.0425 - val_loss: 0.0146 - val_my_metric_fn: 0.0039
Epoch 2/50
2761/2761 [==============================] - 7s 3ms/step - loss: 0.0121 - my_metric_fn: 0.0010 - val_loss: 0.0146 - val_my_metric_fn: 0.0043
Epoch 3/50
2813/2813 [==============================] - 6s 2ms/step - loss: 0.0157 - my_metric_fn: 0.0048
[0.015681525692343712, 0.004816832952201366]
2813/2813 [==============================] - 5s 2ms/step

Epoch 1/50
2761/2761 [==============================] - 9s 3ms/step - loss: 0.0140 - my_metric_fn: -0.1929 - val_loss: 0.0147 - val_my_metric_fn: -0.0065
Epoch 2/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0122 - my_metric_fn: -0.0022 - val_loss: 0.0146 - val_my_metric_fn: 0.0031
Epoch 3/50
2761/2761 [==============================] - 9s 3ms/step - loss: 0.0121 - my_metric_fn: 9.9555e-04 - val_loss: 0.0146 - val_my_metric_fn: 0.0038
Epoch 4/50
2813/2813 [=

2761/2761 [==============================] - 8s 3ms/step - loss: 0.0131 - my_metric_fn: -0.0930 - val_loss: 0.0146 - val_my_metric_fn: -0.0017
Epoch 2/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0121 - my_metric_fn: -9.0620e-04 - val_loss: 0.0146 - val_my_metric_fn: 0.0040
Epoch 3/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0121 - my_metric_fn: 0.0034 - val_loss: 0.0145 - val_my_metric_fn: 0.0061
Epoch 4/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0121 - my_metric_fn: 0.0061 - val_loss: 0.0145 - val_my_metric_fn: 0.0086
Epoch 5/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0120 - my_metric_fn: 0.0126 - val_loss: 0.0143 - val_my_metric_fn: 0.0186
Epoch 6/50
2761/2761 [==============================] - 7s 3ms/step - loss: 0.0118 - my_metric_fn: 0.0287 - val_loss: 0.0139 - val_my_metric_fn: 0.0362
Epoch 7/50
2761/2761 [==============================] - 9s 3ms/step - loss: 0.0115 - my_metr

In [115]:
import random 

seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    
    autoencoder = Autoencoder()
    autoencoder.fit(merged_train_latent, merged_train_latent,
                epochs = 20, batch_size = 216,
                validation_data = (merged_val_latent, merged_val_latent),
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
 
    model = multi_input_model()
    model.layers[4].set_weights(autoencoder.layers[1].get_weights())
    model.layers[4].trainable = False
    model.fit([merged_train_latent, merged_train_firm], merged_train_y,
                   epochs=50,
                   validation_data=([merged_val_latent, merged_val_firm], merged_val_y),
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(model.evaluate([merged_test_latent, merged_test_firm], merged_test_y))
    y_hat = model.predict([merged_test_latent, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

Epoch 1/20
410/410 [==============================] - 2s 3ms/step - loss: 0.2408 - my_metric_fn: -377.7255 - val_loss: 0.2217 - val_my_metric_fn: -475.6530
Epoch 2/20
410/410 [==============================] - 1s 3ms/step - loss: 0.2037 - my_metric_fn: -318.0138 - val_loss: 0.1864 - val_my_metric_fn: -399.7114
Epoch 3/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1700 - my_metric_fn: -265.5910 - val_loss: 0.1548 - val_my_metric_fn: -331.3271
Epoch 4/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1401 - my_metric_fn: -219.0837 - val_loss: 0.1272 - val_my_metric_fn: -271.7142
Epoch 5/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1143 - my_metric_fn: -177.8759 - val_loss: 0.1037 - val_my_metric_fn: -221.1305
Epoch 6/20
410/410 [==============================] - 1s 3ms/step - loss: 0.0926 - my_metric_fn: -144.0566 - val_loss: 0.0843 - val_my_metric_fn: -179.0626
Epoch 7/20
410/410 [==============================] - 1s 3ms/ste

2813/2813 [==============================] - 5s 2ms/step - loss: 0.0141 - my_metric_fn: 0.0629
[0.014079166576266289, 0.06293687224388123]
2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
410/410 [==============================] - 2s 3ms/step - loss: 0.2372 - my_metric_fn: -369.4849 - val_loss: 0.2182 - val_my_metric_fn: -468.0688
Epoch 2/20
410/410 [==============================] - 1s 3ms/step - loss: 0.2003 - my_metric_fn: -313.4544 - val_loss: 0.1831 - val_my_metric_fn: -392.5227
Epoch 3/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1668 - my_metric_fn: -259.2080 - val_loss: 0.1517 - val_my_metric_fn: -324.7347
Epoch 4/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1371 - my_metric_fn: -213.4097 - val_loss: 0.1244 - val_my_metric_fn: -265.8521
Epoch 5/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1117 - my_metric_fn: -173.8555 - val_loss: 0.1013 - val_my_metric_fn: -216.0561
Epoch 6/20
410/410 [===

410/410 [==============================] - 2s 4ms/step - loss: 0.2268 - my_metric_fn: -354.3300 - val_loss: 0.2081 - val_my_metric_fn: -446.4465
Epoch 2/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1907 - my_metric_fn: -297.3618 - val_loss: 0.1741 - val_my_metric_fn: -373.1098
Epoch 3/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1584 - my_metric_fn: -246.6116 - val_loss: 0.1440 - val_my_metric_fn: -308.1270
Epoch 4/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1301 - my_metric_fn: -203.8826 - val_loss: 0.1180 - val_my_metric_fn: -252.0502
Epoch 5/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1059 - my_metric_fn: -164.1261 - val_loss: 0.0961 - val_my_metric_fn: -204.8013
Epoch 6/20
410/410 [==============================] - 1s 3ms/step - loss: 0.0858 - my_metric_fn: -133.3758 - val_loss: 0.0780 - val_my_metric_fn: -165.6928
Epoch 7/20
410/410 [==============================] - 1s 3ms/step - loss: 0

2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
410/410 [==============================] - 2s 3ms/step - loss: 0.2442 - my_metric_fn: -379.5978 - val_loss: 0.2247 - val_my_metric_fn: -482.3583
Epoch 2/20
410/410 [==============================] - 1s 3ms/step - loss: 0.2069 - my_metric_fn: -322.0428 - val_loss: 0.1891 - val_my_metric_fn: -405.7001
Epoch 3/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1728 - my_metric_fn: -270.2353 - val_loss: 0.1571 - val_my_metric_fn: -336.6124
Epoch 4/20
410/410 [==============================] - 2s 4ms/step - loss: 0.1426 - my_metric_fn: -221.6473 - val_loss: 0.1291 - val_my_metric_fn: -276.2031
Epoch 5/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1164 - my_metric_fn: -180.7822 - val_loss: 0.1053 - val_my_metric_fn: -224.8384
Epoch 6/20
410/410 [==============================] - 2s 4ms/step - loss: 0.0944 - my_metric_fn: -146.6658 - val_loss: 0.0855 - val_my_metric_fn: -182.0678
Epoch 

410/410 [==============================] - 2s 3ms/step - loss: 0.2231 - my_metric_fn: -348.3212 - val_loss: 0.2043 - val_my_metric_fn: -438.3093
Epoch 2/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1869 - my_metric_fn: -291.0614 - val_loss: 0.1701 - val_my_metric_fn: -364.6777
Epoch 3/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1546 - my_metric_fn: -241.1662 - val_loss: 0.1401 - val_my_metric_fn: -299.7751
Epoch 4/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1264 - my_metric_fn: -196.7190 - val_loss: 0.1143 - val_my_metric_fn: -244.2092
Epoch 5/20
410/410 [==============================] - 1s 3ms/step - loss: 0.1026 - my_metric_fn: -159.0906 - val_loss: 0.0928 - val_my_metric_fn: -197.7342
Epoch 6/20
410/410 [==============================] - 1s 3ms/step - loss: 0.0828 - my_metric_fn: -128.1853 - val_loss: 0.0751 - val_my_metric_fn: -159.5224
Epoch 7/20
410/410 [==============================] - 1s 3ms/step - loss: 0

In [116]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.1458439875976597


---

In [117]:
# Import Dataset
firm_df = pd.read_pickle('firm_df49.pkl')
firm_df.index = pd.to_datetime(firm_df.index).to_period('M')

# macro-economic variables
# There is one features which has 37 missing values but before year 1980
macro_df = pd.read_pickle('macro_df.pkl')
macro_df.index = pd.to_datetime(macro_df.index, format='%m/%d/%Y')
macro_df.rename(columns={"ep": "ep_macro"}, inplace=True)
macro_index = macro_df.index[1:]
macro_new = macro_df.iloc[:-1, :]
macro_new.index = macro_index
macro_new.index = macro_new.index.to_period('M')

# Scale macro-features
from sklearn.preprocessing import MinMaxScaler
#scaler = MinMaxScaler(feature_range=(-0.5, 0.5))
#macro_scaled = scaler.fit_transform(macro_new)
#macro_scaled_df = pd.DataFrame(macro_scaled, columns=macro_new.columns)
#macro_scaled_df.index = macro_new.index

# merged data 1
merged_df = pd.merge(firm_df, macro_new, left_on='jdate', right_on='sasdate', how='inner')
merged_df.index = firm_df.index

In [118]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
       'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
       'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
       'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
       'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']

info_list = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
       'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

macro_col = ['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx','INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S','IPFUELS', 'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV',
       'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV','UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD',
       'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP','SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT',
       'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE','HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'AMDMNOx', 'ANDENOx', 'AMDMUOx', 'BUSINVx','ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500','S&P: indust', 'S&P div yield', 'S&P PE ratio', 'FEDFUNDS',
       'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA','COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM',
       'AAAFFM', 'BAAFFM', 'TWEXAFEGSMTHx', 'EXSZUSx', 'EXJPUSx','EXUSUKx', 'EXCAUSx', 'WPSFD49207', 'WPSFD49502', 'WPSID61',
       'WPSID62', 'OILPRICEx', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL','CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD',
       'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5','PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA',
       'CES0600000008', 'CES2000000008', 'CES3000000008', 'UMCSENTx','DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'VIXCLSx', 'dp', 'ep_macro', 'b/m',
       'ntis', 'tbl', 'tms', 'dfy', 'svar']

macro_core =  ['dp', 'ep_macro', 'b/m','ntis', 'tbl', 'tms', 'dfy', 'svar']

def datasplit(df, dataset):
    df_train = df[df.index < pd.Period((str(1995)+"-1"),freq='M')]
    df_val = df[(df.index >= pd.Period((str(1995)+"-1"),freq='M')) & (df.index < pd.Period((str(2000)+"-1"),freq='M'))]
    df_test = df[df.index >= pd.Period((str(2000)+"-1"),freq='M')]
    if dataset == "firm":
        X_train = df_train[char_core]
        X_val = df_val[char_core]
        X_test = df_test[char_core]
    elif dataset == 'macro':
        X_train = df_train[macro_col]
        X_val = df_val[macro_col]
        X_test = df_test[macro_col]
    elif dataset == "firm_macro":
        X_train = df_train[char_core+macro_col]
        X_val = df_val[char_core+macro_col]
        X_test = df_test[char_core+macro_col]
    elif dataset == "firm_macro8":
        X_train = df_train[char_core+macro_core]
        X_val = df_val[char_core+macro_core]
        X_test = df_test[char_core+macro_core] 
    y_train = df_train['predicted_return']
    y_val = df_val['predicted_return']
    y_test = df_test['predicted_return']
    return (X_train, y_train, X_val, y_val, X_test, y_test, )

In [119]:
merged_df.columns.tolist()[17:]
# merged data 1
firm_col = merged_df.columns.tolist()[17:17+49]
latent_col = merged_df.columns.tolist()[17+49+1:]
print(len(firm_col), len(latent_col))

49 134


In [120]:
merged_latent = merged_df.copy()
df_train = merged_latent[merged_latent.index < pd.Period((str(1995)+"-1"),freq='M')]
df_val = merged_latent[(merged_latent.index >= pd.Period((str(1995)+"-1"),freq='M')) & (merged_latent.index < pd.Period((str(2000)+"-1"),freq='M'))]
df_test = merged_latent[merged_latent.index >= pd.Period((str(2000)+"-1"),freq='M')]

merged_train_firm = df_train[firm_col]
merged_train_latent = df_train[latent_col]
merged_train_y = df_train['predicted_return']
print(merged_train_firm.shape, merged_train_latent.shape, merged_train_y.shape)

merged_val_firm = df_val[firm_col]
merged_val_latent = df_val[latent_col]
merged_val_y = df_val['predicted_return']
print(merged_val_firm.shape, merged_val_latent.shape, merged_val_y.shape)

merged_test_firm = df_test[firm_col]
merged_test_latent = df_test[latent_col]
merged_test_y = df_test['predicted_return']
print(merged_test_firm.shape, merged_test_latent.shape, merged_test_y.shape)

(88347, 49) (88347, 134) (88347,)
(31851, 49) (31851, 134) (31851,)
(90007, 49) (90007, 134) (90007,)


In [125]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

def multi_input_model():
    l1_reg = 0.0
    l2_reg = 0.0
    dropout = 0.0
    lr = 0.0001
        
    # Define the hidden layers 
    input1 = Input(shape=(134,))
    x1 = Dense(6, activation='relu')(input1)
        
    # Input 2
    input2 = Input(shape=(49,))
    x2 = Dense(32, activation='relu')(input2)
    x2 = Dense(16, activation='relu')(x2)
    #x2 = Dense(8, activation='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(x2)
    x2 = Dense(6, activation='relu')(x2)
        
    merged = Multiply()([x1, x2])
    output = Dense(1, activation='linear')(merged)
 
    # Create the model 
    model = Model(inputs=[input1, input2], outputs=output)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
    return model

In [126]:
def Autoencoder():
    input_img = Input(shape=(134, ))
    encoded1 = Dense(6, activation='sigmoid')(input_img)
    decoded1 = Dense(134, activation='sigmoid')(encoded1)
    
    autoencoder = Model(inputs=input_img, outputs=decoded1)
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    autoencoder.compile(loss='mean_squared_error', optimizer=opt, metrics=[my_metric_fn])
    return autoencoder

autoencoder = Autoencoder()

In [127]:
import random 

seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)
    
    autoencoder = Autoencoder()
    autoencoder.fit(merged_train_latent, merged_train_latent,
                epochs = 20, batch_size = 256,
                validation_data = (merged_val_latent, merged_val_latent),
                 callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
 
    model = multi_input_model()
    model.layers[4].set_weights(autoencoder.layers[1].get_weights())
    model.layers[4].trainable = False
    model.fit([merged_train_latent, merged_train_firm], merged_train_y,
                   epochs=50,
                   validation_data=([merged_val_latent, merged_val_firm], merged_val_y),
                   callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss',mode='min', patience=1)])
    print(model.evaluate([merged_test_latent, merged_test_firm], merged_test_y))
    y_hat = model.predict([merged_test_latent, merged_test_firm]).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

Epoch 1/20
410/410 [==============================] - 2s 5ms/step - loss: 139.4794 - my_metric_fn: 0.0086 - val_loss: 272.2100 - val_my_metric_fn: 0.0126
Epoch 2/20
410/410 [==============================] - 2s 4ms/step - loss: 139.1864 - my_metric_fn: 0.0107 - val_loss: 271.9395 - val_my_metric_fn: 0.0150
Epoch 3/20
410/410 [==============================] - 2s 4ms/step - loss: 138.9742 - my_metric_fn: 0.0122 - val_loss: 271.7515 - val_my_metric_fn: 0.0167
Epoch 4/20
410/410 [==============================] - 2s 4ms/step - loss: 138.8177 - my_metric_fn: 0.0133 - val_loss: 271.6062 - val_my_metric_fn: 0.0181
Epoch 5/20
410/410 [==============================] - 2s 5ms/step - loss: 138.6937 - my_metric_fn: 0.0142 - val_loss: 271.4890 - val_my_metric_fn: 0.0191
Epoch 6/20
410/410 [==============================] - 2s 5ms/step - loss: 138.5941 - my_metric_fn: 0.0150 - val_loss: 271.3937 - val_my_metric_fn: 0.0200
Epoch 7/20
410/410 [==============================] - 2s 4ms/step - loss: 13

Epoch 5/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0121 - my_metric_fn: -4.6103e-04 - val_loss: 0.0146 - val_my_metric_fn: 0.0030
Epoch 6/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0121 - my_metric_fn: 0.0013 - val_loss: 0.0146 - val_my_metric_fn: 0.0041
Epoch 7/50
2813/2813 [==============================] - 6s 2ms/step - loss: 0.0158 - my_metric_fn: -0.0134
[0.015801146626472473, -0.013388097286224365]
2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
410/410 [==============================] - 2s 4ms/step - loss: 139.4989 - my_metric_fn: 0.0084 - val_loss: 272.1981 - val_my_metric_fn: 0.0125
Epoch 2/20
410/410 [==============================] - 1s 3ms/step - loss: 139.2275 - my_metric_fn: 0.0104 - val_loss: 271.9663 - val_my_metric_fn: 0.0147
Epoch 3/20
410/410 [==============================] - 1s 3ms/step - loss: 139.0273 - my_metric_fn: 0.0118 - val_loss: 271.7787 - val_my_metric_fn: 0.0165
Epoch 4/20
410/410

2761/2761 [==============================] - 8s 3ms/step - loss: 0.0189 - my_metric_fn: -0.6654 - val_loss: 0.0211 - val_my_metric_fn: -0.7267
Epoch 3/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0130 - my_metric_fn: -0.0880 - val_loss: 0.0163 - val_my_metric_fn: -0.2253
Epoch 4/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0122 - my_metric_fn: -0.0110 - val_loss: 0.0152 - val_my_metric_fn: -0.1054
Epoch 5/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0121 - my_metric_fn: 0.0012 - val_loss: 0.0152 - val_my_metric_fn: -0.1014
Epoch 6/50
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0171 - my_metric_fn: -0.1244
[0.01707121729850769, -0.12440617382526398]
2813/2813 [==============================] - 5s 2ms/step

Epoch 1/20
410/410 [==============================] - 2s 4ms/step - loss: 139.5035 - my_metric_fn: 0.0084 - val_loss: 272.2508 - val_my_metric_fn: 0.0114
Epoch 2/20
410/410 [=============

2761/2761 [==============================] - 8s 3ms/step - loss: 1.2007 - my_metric_fn: -125.1322 - val_loss: 0.0381 - val_my_metric_fn: -2.3418
Epoch 2/50
2761/2761 [==============================] - 7s 3ms/step - loss: 0.0164 - my_metric_fn: -0.4234 - val_loss: 0.0171 - val_my_metric_fn: -0.2553
Epoch 3/50
2761/2761 [==============================] - 10s 3ms/step - loss: 0.0125 - my_metric_fn: -0.0411 - val_loss: 0.0151 - val_my_metric_fn: -0.0469
Epoch 4/50
2761/2761 [==============================] - 9s 3ms/step - loss: 0.0122 - my_metric_fn: -0.0041 - val_loss: 0.0147 - val_my_metric_fn: -0.0113
Epoch 5/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0121 - my_metric_fn: 0.0014 - val_loss: 0.0147 - val_my_metric_fn: -0.0017
Epoch 6/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0121 - my_metric_fn: 0.0021 - val_loss: 0.0146 - val_my_metric_fn: -9.9964e-04
Epoch 7/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0121

410/410 [==============================] - 1s 3ms/step - loss: 138.0275 - my_metric_fn: 0.0190 - val_loss: 270.6721 - val_my_metric_fn: 0.0248
Epoch 17/20
410/410 [==============================] - 1s 3ms/step - loss: 137.9836 - my_metric_fn: 0.0193 - val_loss: 270.6175 - val_my_metric_fn: 0.0252
Epoch 18/20
410/410 [==============================] - 1s 3ms/step - loss: 137.9426 - my_metric_fn: 0.0196 - val_loss: 270.5671 - val_my_metric_fn: 0.0255
Epoch 19/20
410/410 [==============================] - 1s 3ms/step - loss: 137.9044 - my_metric_fn: 0.0198 - val_loss: 270.5206 - val_my_metric_fn: 0.0257
Epoch 20/20
410/410 [==============================] - 1s 3ms/step - loss: 137.8693 - my_metric_fn: 0.0201 - val_loss: 270.4781 - val_my_metric_fn: 0.0260
Epoch 1/50
2761/2761 [==============================] - 9s 3ms/step - loss: 0.1002 - my_metric_fn: -8.9482 - val_loss: 0.0167 - val_my_metric_fn: -0.1966
Epoch 2/50
2761/2761 [==============================] - 8s 3ms/step - loss: 0.0127 

410/410 [==============================] - 3s 7ms/step - loss: 138.2914 - my_metric_fn: 0.0170 - val_loss: 270.9831 - val_my_metric_fn: 0.0229
Epoch 10/20
410/410 [==============================] - 3s 7ms/step - loss: 138.2167 - my_metric_fn: 0.0176 - val_loss: 270.8987 - val_my_metric_fn: 0.0234
Epoch 11/20
410/410 [==============================] - 2s 4ms/step - loss: 138.1501 - my_metric_fn: 0.0180 - val_loss: 270.8226 - val_my_metric_fn: 0.0239
Epoch 12/20
410/410 [==============================] - 1s 3ms/step - loss: 138.0910 - my_metric_fn: 0.0186 - val_loss: 270.7552 - val_my_metric_fn: 0.0243
Epoch 13/20
410/410 [==============================] - 1s 3ms/step - loss: 138.0382 - my_metric_fn: 0.0189 - val_loss: 270.6943 - val_my_metric_fn: 0.0247
Epoch 14/20
410/410 [==============================] - 1s 3ms/step - loss: 137.9907 - my_metric_fn: 0.0192 - val_loss: 270.6393 - val_my_metric_fn: 0.0251
Epoch 15/20
410/410 [==============================] - 1s 3ms/step - loss: 137.948

In [128]:
# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = merged_test_y

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

-0.5260050936248688
